In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
 
#폰트 설정          
from matplotlib import rc  
%matplotlib inline           
rc('font', family='AppleGothic' )
plt.rcParams['axes.unicode_minus'] = False  

import lightgbm as lgb

OSError: dlopen(/Users/yuchanghee/opt/anaconda3/envs/dacon/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/yuchanghee/opt/anaconda3/envs/dacon/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [16]:
#전처리 함수
def prepro():   
    #데이터 불러오기
    dir='/Users/yuchanghee/Desktop/DACON/Parking_Demand/data/'
    train= pd.read_csv(dir+'train.csv') 
    test= pd.read_csv(dir+'test.csv') 
    age_gender_info=pd.read_csv(dir+'age_gender_info.csv')
    sample_submission=pd.read_csv(dir+'sample_submission.csv')
    train=train.rename({'도보 10분거리 내 지하철역 수(환승노선 수 반영)': '지하철', '도보 10분거리 내 버스정류장 수': '버스'},axis=1)
    test=test.rename({'도보 10분거리 내 지하철역 수(환승노선 수 반영)': '지하철', '도보 10분거리 내 버스정류장 수': '버스'},axis=1)
    age_gender_info=age_gender_info.set_index('지역')
    age_gender_info.loc['전체 분포 평균']=age_gender_info.mean()        
    age_gender_info.loc['광역시 평균']=age_gender_info[age_gender_info.index.str.contains('시')].mean()
    age_gender_info.loc['도 평균']=age_gender_info[age_gender_info.index.str.contains('도')].mean()

    #데이터 concat, 중복제거 
    total=pd.concat([train,test],axis=0) 
    total.reset_index(drop=True, inplace=True) 
    total=total.drop_duplicates().reset_index(drop=True) 

    #null 처리
    #1. 자격유형 null
    total.loc[(total['자격유형'].isnull()) & (total['단지코드']=='C2411'), '자격유형']='A'
    total.loc[(total['자격유형'].isnull()) & (total['단지코드']=='C2253'), '자격유형']='C'

    #2. 임대료, 임대보증금 null 
    total['임대료'].replace('-',np.nan,inplace=True)  
    total['임대보증금'].replace('-',np.nan,inplace=True)
    total['임대료']=total['임대료'].astype(np.float64)   
    total['임대보증금']=total['임대보증금'].astype(np.float64) 

    #2.1 아파트 - 지역과 공급유형에 따라 임대료, 임대보증금 처리
    total.loc[total['공급유형']=='장기전세', '임대료']=0 #장기전세는 장기전세는 임대료가 없으므로 0
    total.loc[2707,'임대보증금']=5787000.0 
    total.loc[2707,'임대료']=79980.0 
    total.loc[2709,'임대보증금']=11574000.0 
    total.loc[2709,'임대료']=159960.0
    total.loc[(total['지역']=='대전광역시') & (total['공급유형']=='공공분양'), ['임대료','임대보증금']]=0  #공공분양의 경우 보증금, 임대료 없으므로 총분양가 변수 생성 
    total['총분양가']=0.0
    total.loc[(total['지역']=='대전광역시') & (total['공급유형']=='공공분양'), '총분양가']=[221640000, 217200000, 221640000, 251220000, 251220000, 251260000, 251320000]
    total.loc[(total['임대건물구분']=='아파트') & (total['임대보증금'].isnull()) & (total['지역'] == '부산광역시'), '임대보증금'] = [7000000,7000000,7600000,14800000,23100000]  
    total.loc[(total['임대건물구분']=='아파트') & (total['임대료'].isnull()) & (total['지역'] == '부산광역시'), '임대료']= [135000, 135000, 142000, 198000, 259000]    
    total.loc[(total['임대건물구분']=='아파트') & (total['임대보증금'].isnull()) & (total['지역'] == '대구광역시'), '임대보증금'] = [10847000, 10847000, 17338000] 
    total.loc[(total['임대건물구분']=='아파트') & (total['임대료'].isnull()) & (total['지역'] == '대구광역시'), '임대료'] = [138600, 138600, 197500]  
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='행복주택') & (total['임대보증금'].isnull()) & (total['전용면적']==16.94) , '임대보증금']=14965000.0 
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='행복주택') & (total['임대료'].isnull()) & (total['전용면적']==16.94) , '임대료']=70915.0 
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='행복주택') & (total['임대보증금'].isnull()) & (total['전용면적']==36.770) , '임대보증금']=34140000.000
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='행복주택') & (total['임대료'].isnull()) & (total['전용면적']==36.770) , '임대료']=167000.000  
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='행복주택') & (total['임대보증금'].isnull()) & (total['전용면적']==26.850) , '임대보증금'] = 25000000.000 
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='행복주택') & (total['임대료'].isnull()) & (total['전용면적']==26.850) , '임대료'] = 130000.000 
    total.loc[(total['지역']=='강원도') & (total['공급유형']=='행복주택') & total['임대보증금'].isnull() , '임대보증금']= [13181000, 19700000, 19150000, 21679000]  
    total.loc[(total['지역']=='강원도') & (total['공급유형']=='행복주택') & total['임대료'].isnull() , '임대료'] = [65500, 96000, 94000, 105000] 
    total.loc[(total['지역']=='강원도') & (total['공급유형']=='영구임대') & total['임대보증금'].isnull(), '임대보증금']=[2129000, 2902000]
    total.loc[(total['지역']=='강원도') & (total['공급유형']=='영구임대') & total['임대료'].isnull(), '임대료']=[42350, 57730]

    #2.2 상가 
    total['임대보증금'] = total['임대보증금'].fillna(0)
    total['임대료'] = total['임대료'].fillna(0)
  
    #3. 버스 - 임대보증금, 등록차량수와 상관성이 있으므로 그에 따라 대체 
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='공공임대(10년)') & (total['지하철'].isnull()) , '지하철']=0
    total.loc[(total['지역']=='경상남도') & (total['공급유형']=='공공임대(10년)') &(total['버스'].isnull()), '버스'] = 3

    #4. 지하철 - 일괄적으로 0처리 
    total['지하철']=total['지하철'].fillna(0)  
    total['지하철'].value_counts(dropna=False)
    
    #성별, 세대 merge
    total_age=total.merge(age_gender_info.reset_index(), on='지역', how='inner') 
    return train, test, total, total_age

In [17]:
train, test,total, total_age=prepro()

In [18]:
total.isnull().sum()  

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금         0
임대료           0
지하철           0
버스            0
단지내주차면수       0
등록차량수       949
총분양가          0
dtype: int64

In [21]:
total_age

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000.0,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000.0,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000.0,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000.0,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000.0,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3576,C1229,453,아파트,세종특별자치시,행복주택,38.20,68,23.0,J,30101000.0,...,0.065279,0.051163,0.026056,0.026223,0.013893,0.007086,0.003459,0.001004,0.000279,0.000112
3577,C1229,453,아파트,세종특별자치시,행복주택,38.20,8,23.0,J,30101000.0,...,0.065279,0.051163,0.026056,0.026223,0.013893,0.007086,0.003459,0.001004,0.000279,0.000112
3578,C1229,453,아파트,세종특별자치시,행복주택,38.25,3,23.0,J,30101000.0,...,0.065279,0.051163,0.026056,0.026223,0.013893,0.007086,0.003459,0.001004,0.000279,0.000112
3579,C1229,453,아파트,세종특별자치시,행복주택,38.28,4,23.0,J,30101000.0,...,0.065279,0.051163,0.026056,0.026223,0.013893,0.007086,0.003459,0.001004,0.000279,0.000112


In [22]:
#<원핫인코딩>
total_age_ohe=pd.get_dummies(total_age)
print(total_age_ohe.shape)   
print('object타입:')
print(total_age_ohe.dtypes[total_age_ohe.dtypes=='object'])          

(3581, 649)
object타입:
Series([], dtype: object)


In [25]:
X_train = total_age[total_age['등록차량수'].notnull()]

In [36]:
import lightgbm

OSError: dlopen(/Users/yuchanghee/opt/anaconda3/envs/dacon/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/yuchanghee/opt/anaconda3/envs/dacon/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [31]:
 
reg = setup(X_train, 
            preprocess = False, # True로 설정되면, 자체적인 Feature Engineering을 추가로 진행해 Predict가 불가능해진다.
            train_size = 0.999,  # 우리는 전체 데이터를 학습해 test를 예측하는게 목표이기 때문에, 0.999로 설정한다.
            target = '등록차량수', # 목표 변수는 등록 차량 수 이다.
            silent = True, # 엔터를 누르기 귀찮다. 궁금하면 풀어보세요
            use_gpu = False, # GPU가 있으면 사용하세요 (Cat BOost 속도 향상)
            numeric_features=list(X_train.drop(columns = ['등록차량수']).columns), # 모든 변수가 숫자로써의 의미가 있다.
            session_id = 2021,
            fold_shuffle = True
            )

OSError: dlopen(/Users/yuchanghee/opt/anaconda3/envs/dacon/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/yuchanghee/opt/anaconda3/envs/dacon/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [28]:
!pip install pycaret 

     |████████████████████████████████| 264 kB 1.6 MB/s 
     |████████████████████████████████| 29.1 MB 2.8 MB/s 
  Using cached ipywidgets-7.6.3-py2.py3-none-any.whl (121 kB)
     |████████████████████████████████| 105 kB 3.8 MB/s 
     |████████████████████████████████| 1.3 MB 3.8 MB/s 
     |████████████████████████████████| 80 kB 4.2 MB/s 
     |████████████████████████████████| 81 kB 4.3 MB/s 
     |████████████████████████████████| 1.5 MB 4.0 MB/s 
     |████████████████████████████████| 636 kB 4.1 MB/s 
     |████████████████████████████████| 10.1 MB 6.6 MB/s 
     |████████████████████████████████| 23.4 MB 9.2 MB/s 
  Using cached lightgbm-3.2.1-py3-none-macosx_10_14_x86_64.macosx_10_15_x86_64.macosx_11_0_x86_64.whl (1.2 MB)
     |████████████████████████████████| 220 kB 12.6 MB/s 
     |████████████████████████████████| 56 kB 10.7 MB/s 
     |████████████████████████████████| 1.7 MB 12.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... 

In [ ]:
new_total = pd.DataFrame()
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = total[total['단지코드']==code]
    temp.index = range(temp.shape[0])

    for col in temp.columns.drop(['전용면적','전용면적별세대수','임대보증금','임대료','총분양가']):
        new_total.loc[i, col] = temp.loc[0, col] 
        
    for col in ['전용면적','전용면적별세대수','임대보증금','임대료','총분양가']:   
        for j in temp['전용면적'].unique().tolist():
            new_total.loc[i, '전용면적_'+str(j)] = temp[temp['전용면적']==j]['전용면적별세대수'].sum()
        temp['전용면적'] 
        temp[temp['전용면적']==area]
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum() 
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']

In [10]:
#상가 null 처리
#해당 지역   
total[total['임대보증금'].isnull()]['지역'].value_counts()    

부산광역시      152
대전광역시       71
충청남도        59
경상남도        45
강원도         39
울산광역시       28
제주특별자치도      3
Name: 지역, dtype: int64

In [20]:
#1. 부산광역시 
total[(total['지역']=='부산광역시') & (total['임대보증금'].isnull())]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수,총분양가
271,C2135,1116,상가,부산광역시,임대상가,14.10,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
272,C2135,1116,상가,부산광역시,임대상가,18.98,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
273,C2135,1116,상가,부산광역시,임대상가,19.36,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
274,C2135,1116,상가,부산광역시,임대상가,21.46,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
275,C2135,1116,상가,부산광역시,임대상가,22.83,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2761,C2676,2572,상가,부산광역시,임대상가,54.60,1,25.0,D,NaN,NaN,0.0,2.0,418.0,NaN,0.0
2762,C2676,2572,상가,부산광역시,임대상가,54.61,1,25.0,D,NaN,NaN,0.0,2.0,418.0,NaN,0.0
2763,C2676,2572,상가,부산광역시,임대상가,75.98,1,25.0,D,NaN,NaN,0.0,2.0,418.0,NaN,0.0
2764,C2676,2572,상가,부산광역시,임대상가,404.65,1,25.0,D,NaN,NaN,0.0,2.0,418.0,NaN,0.0


In [17]:
total[(total['지역']=='부산광역시') & (total['임대보증금'].isnull())]['단지코드'].value_counts()              

C2132    27
C2676    27
C1439    16
C2034    15
C1109    15
C2597    15
C2135    13
C2289    12
C2310    12
Name: 단지코드, dtype: int64

In [21]:
total[total['단지코드']=='C2135']  #부산금곡1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수,총분양가
268,C2135,1116,아파트,부산광역시,영구임대,26.37,358,10.0,C,9363000.0,124390.0,1.0,3.0,262.0,127.0,0.0
269,C2135,1116,아파트,부산광역시,영구임대,26.37,294,10.0,C,9363000.0,124390.0,1.0,3.0,262.0,127.0,0.0
270,C2135,1116,아파트,부산광역시,영구임대,31.32,447,10.0,C,10692000.0,137660.0,1.0,3.0,262.0,127.0,0.0
271,C2135,1116,상가,부산광역시,임대상가,14.10,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
272,C2135,1116,상가,부산광역시,임대상가,18.98,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
273,C2135,1116,상가,부산광역시,임대상가,19.36,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
274,C2135,1116,상가,부산광역시,임대상가,21.46,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
275,C2135,1116,상가,부산광역시,임대상가,22.83,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
276,C2135,1116,상가,부산광역시,임대상가,23.40,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0
277,C2135,1116,상가,부산광역시,임대상가,27.55,1,10.0,D,NaN,NaN,1.0,3.0,262.0,127.0,0.0


In [24]:
total.loc[(total['단지코드']=='C2135') & (total['전용면적']==23.4), '임대료']=265000 
total.loc[(total['단지코드']=='C2135') & (total['전용면적']==23.4), '임대보증금']=6360000  
 
total.loc[(total['단지코드']=='C2135') & (total['전용면적']==18.98), '임대료']=133000
total.loc[(total['단지코드']=='C2135') & (total['전용면적']==18.98), '임대보증금']=3192000

     
 

276   NaN
Name: 임대료, dtype: float64

In [26]:
total[(total['단지코드']=='C2135') & (total['공급유형']=='임대상가')]['전용면적'].value_counts()

14.10     1
18.98     1
19.36     1
21.46     1
22.83     1
23.40     1
27.55     1
29.17     1
31.92     1
32.60     1
36.57     1
50.08     1
240.22    1
Name: 전용면적, dtype: int64